To run with higer concurrency you'll need to scale up the VM.  By default it runs with 4 cores.  To add a code cell below this that looks like:

```
%%configure -f
{
    "vCores": 16,// Recommended values: [4, 8, 16, 32, 64], Fabric will allocate matched memory according to the specified vCores.
}
```

Note that this will increase the notebook startup time as a VM will need to be spun-up.


%%configure -f
{
    "vCores": 4
}

In [ ]:
import uuid
import sempy.fabric
import pandas as pd
import polars as pl
import os
from deltalake import DeltaTable
from deltalake.writer import write_deltalake
import sempy
import time 



load_test_name = "<load test name here>"                                                    #name of test for logging
dataset = "<semantic model name here>"                                                      #name of the semantic model to test
workspace = None                                                           #workspace containing the semantic model to test "None" for the current workspace
queryfile = "/lakehouse/default/Files/PerfScenarios/Queries/PowerBIPerformanceData.json"    #Power BI Desktop performance analyzer file containing the use case to test
concurrent_threads = 6                                                                      #number of concurrent threads to test
delay_sec = 4                                                                               #number of seconds each user waits between iterations of the use case
iterations = 1                                                                              #number of iterations run by each user

workspace = workspace if workspace is not None else notebookutils.runtime.context['currentWorkspaceName']

if 'Collapse Here':
    ts=time.strftime("%Y%m%d-%H%M%S")
    loadtestId:str = f"{load_test_name}-{ts}"
    print(loadtestId)

    lakehouseId = sempy.fabric.get_lakehouse_id()
    workspaceId = sempy.fabric.get_workspace_id()

    # Path for storing CSV files during run
    folder_path = f"/lakehouse/default/Files/PerfScenarios/logs/{loadtestId}"
    # Used for writing combined logs


    #Create folder dedicated to logs from this run only
    notebookutils.fs.mkdirs(folder_path)


    users = []      # example when populated could be 
                    # 
                    # users = ["user1@domain.com","user@domain.com","user3@domain.com"]
                

    args = { 
        "xmla_endpoint" : None, #defaults to current workspace's endpoint
        "workspace" : workspace,
        "perf_analyzer_filename" : queryfile , 
        "model" : dataset,
        "roles" : None,
        "customdata" : None,
        "effective_username" : None,
        "iterations" : iterations,
        "delay_sec" : 1,
        "loadtestId" : loadtestId ,
        "threadId" : 0 ,
        "concurrent_threads" : concurrent_threads,
        "useRootDefaultLakehouse": True
    }

    activity =  {
                "name": "RunPerfScenario", # activity name, must be unique
                "path": "RunPerfScenario", # notebook path
                "timeoutPerCellInSeconds": 9000, # max timeout for each cell, default to 90 seconds
                "args": {"param1": "value1"}, # notebook parameters
                "workspace": None, # workspace name, default to current workspace
                "retry": 0, # max retry times, default to 0
                "retryIntervalInSeconds": 0, # retry interval, default to 0 seconds
                "dependencies": [] # list of activity names that this activity depends on
            }


    DAG = {
        "activities": [],
        "timeoutInSeconds": 43200, # max timeout for the entire pipeline, default to 12 hours
        "concurrency": 25 # max number of notebooks to run concurrently, default to 25
    }
    DAG["concurrency"] = concurrent_threads

    for i in range(concurrent_threads):
        a = activity.copy()
        a["name"] = f"{a['name']}_{i}"
        this_args = args.copy()
        if (len(users)>0):
            user = users[i%len(users)]
            this_args["effective_username"] = user 
            #this_args["customdata"] = user

        this_args["threadId"] = i
        a["args"] = this_args

        DAG["activities"].append(a)
        
    # print(DAG)
    print('running load test')
    try:

        ####################################
        ##  Run the parallel threads here ##
        ####################################

        results = notebookutils.notebook.runMultiple(DAG) 


    except Exception as e:
        first_result = results[list(e.result)[0]]
        print('Notebook failed.  First Result Exception: ')
        print(first_result.get('exception',None))
    print('load test complete')

    # Consolodate CSV log files of results into single dataframe
    df_list = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            polars_df = pl.read_csv(file_path)
            # Need to store Time using format that will work for Direct Lake

            polars_df = (polars_df
                .with_columns([
                    pl.col("start_time_dt").str.to_datetime(time_unit="ms").dt.truncate("1s").alias("start_time_s").dt.replace_time_zone(time_zone="UTC")
                    ])
                )

            polars_df = (polars_df
                .with_columns([
                    pl.col("start_time_dt").str.to_datetime(time_unit="ms").alias("start_time_dt").dt.replace_time_zone(time_zone="UTC")
                    ])
                )

            df_list.append(polars_df)

    combined_df = pl.concat(df_list)

In [ ]:
combined_df = combined_df.with_columns(pl.col("start_time_s").cast(pl.Datetime).alias("start_time_s_dt"))


combined_df = combined_df.with_columns(
    combined_df.group_by("start_time_s_dt")
      .agg(pl.col("duration").mean().alias("avg1"))
      .join(combined_df, on="start_time_s_dt")
      .select("avg1")
)

combined_df = combined_df.with_columns(
    combined_df.group_by("start_time_s_dt")
      .agg(pl.col("duration").quantile(0.9).alias("p90"))
      .join(combined_df, on="start_time_s_dt")
      .select("p90")
)

combined_df = combined_df.with_columns(
    combined_df.group_by("start_time_s_dt")
      .agg(pl.col("duration").quantile(0.5).alias("p50"))
      .join(combined_df, on="start_time_s_dt")
      .select("p50")
)

display(combined_df)
